In [4]:
import sys
sys.path.append('../')
import pandas as pd
import numpy as np
import netCDF4
from helper import db
import math


In [39]:
data = netCDF4.Dataset('../data/gtspp_13050025_te_111.nc')
data.variables;

In [49]:
from os import listdir
import os

gulf_coords = [(51.416032, -57.011715), (45.659196, -57.011715), (51.416032, -66.196285), (45.659196, -66.196285) ]
latitude_min, latitude_max = 45.659196, 51.416032 
longitude_min, longitude_max = -66.196285, -57.011715

gstpp_directory='../data/atlantic/2012/08'
latitude, longitude = [], []
for file in os.listdir(gstpp_directory):
    cdf_coordinates = netCDF4.Dataset(f'{gstpp_directory}/{file}')
    print()
    array_lat, array_long = cdf_coordinates.variables['latitude'][:], cdf_coordinates.variables['longitude'][:]
    lat = array_lat[array_lat.mask==False].data[0].tolist()[0]
    long = array_long[array_long.mask==False].data[0].tolist()[0]

    if ((lat >= latitude_min) & (lat <= latitude_max) & (long >= longitude_min) & (long <= longitude_max)):        
        latitude.append(lat)
        longitude.append(long)
    if len(latitude)>=10:
        break
    
coords = pd.DataFrame(list(zip(latitude, longitude)), columns=['latitude', 'longitude']);
# coords = pd.DataFrame(gulf_1, columns=['latitude', 'longitude'])

FileNotFoundError: [WinError 3] The system cannot find the path specified: '../data/atlantic/2012/08'

In [36]:
import pandas as pd
sql = """SELECT longitude, latitude, station_id, measure_time, salinity, depth, temperature FROM gtspp 
WHERE EXTRACT(YEAR FROM measure_time)=2019 AND position_quality=1 
AND measure_time_quality=1 AND salinity_quality=1 AND depth_quality=1 AND temperature_quality=1;"""

good_data_df = db.run_query(sql)
bin_max = (math.ceil(good_data_df['depth'].max()/100)+1)*100
cut_bins = [i for i in range(0, bin_max, 100)]

In [47]:
good_data_df['depth_bin'] = pd.cut(good_data_df['depth'], cut_bins, right=False)
good_data_df['depth_bin'] = good_data_df['depth_bin'].astype('str')
grouped_df = good_data_df.groupby(['measure_time', 'latitude', 'longitude', 'station_id', 'depth_bin'])[['salinity', 'temperature']].mean().reset_index()


RENAME_COLUMNS = {'measure_time': 'data_date', 'station_id': 'floatid', 'depth_bin': 'depth'}
grouped_df.rename(columns=RENAME_COLUMNS, inplace=True)
grouped_df['data_source']='GTSPP'



In [51]:
grouped_df.head()

,data_date,latitude,longitude,floatid,depth,salinity,temperature,data_source
0,2019-03-02 19:24:59.990860,46.8470001221,-71.1157989502,34707183,"[0, 100)",0.120000,0.130000,GTSPP
1,2019-03-02 19:51:59.966150,46.9160003662,-70.8898010254,34707179,"[0, 100)",0.120000,0.100000,GTSPP
2,2019-03-03 13:05:59.980790,47.9192008972,-69.7817993164,34707171,"[0, 100)",28.692353,1.175882,GTSPP
3,2019-03-03 13:05:59.980790,47.9192008972,-69.7817993164,34707171,"[100, 200)",31.500000,1.830000,GTSPP
4,2019-03-03 13:35:59.994499,48.2923011780,-69.2517013550,34707172,"[0, 100)",31.024286,1.799286,GTSPP


In [1]:
# this was testing to see the captured points were within the shape
import folium

import folium
m = folium.Map(location=[50, -60], zoom_start=4.5, tiles='Stamen Terrain')

for flt in df.index[:1]:
    folium.Marker(location=[df["latitude"].iloc[flt], df["longitude"].iloc[flt]]).add_to(m)
m

NameError: name 'df' is not defined